Imports


In [198]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

Load data

In [199]:
data_path = "../udataset/state_data.csv"
# data_path = "../udataset/health_data.csv"
# data_path = "../udataset/principle_components.csv"
df = pd.read_csv(data_path)
df

,state,year,"agriculture, forestry, fishing and hunting, and mining",construction,manufacturing,wholesale trade,retail trade,"transportation and warehousing, and utilities",information,"finance and insurance, and real estate and rental and leasing","professional, scientific, and management, and administrative and waste management services","educational services, and health care and social assistance","arts, entertainment, and recreation, and accommodation and food services","other services, except public administration",public administration
0,alabama,2010,0.142857,0.803030,0.748603,0.787879,0.806452,0.370968,0.25000,0.328571,0.144330,0.440559,0.031579,0.196429,0.161972
1,alabama,2011,0.142857,0.742424,0.731844,0.727273,0.806452,0.354839,0.21875,0.314286,0.154639,0.482517,0.036842,0.214286,0.176056
2,alabama,2012,0.134921,0.696970,0.709497,0.696970,0.817204,0.370968,0.21875,0.300000,0.159794,0.503497,0.047368,0.214286,0.183099
3,alabama,2013,0.134921,0.651515,0.703911,0.636364,0.806452,0.354839,0.18750,0.300000,0.164948,0.538462,0.057895,0.214286,0.183099
4,alabama,2014,0.134921,0.606061,0.709497,0.636364,0.817204,0.338710,0.18750,0.300000,0.164948,0.545455,0.068421,0.214286,0.190141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,wyoming,2018,0.904762,0.818182,0.167598,0.454545,0.698925,0.532258,0.12500,0.071429,0.041237,0.636364,0.194737,0.071429,0.211268
672,wyoming,2019,0.865079,0.848485,0.167598,0.424242,0.698925,0.516129,0.12500,0.085714,0.046392,0.685315,0.184211,0.107143,0.190141
673,wyoming,2020,0.825397,0.848485,0.178771,0.424242,0.677419,0.532258,0.15625,0.142857,0.030928,0.741259,0.147368,0.107143,0.197183
674,wyoming,2021,0.793651,0.863636,0.173184,0.393939,0.731183,0.516129,0.12500,0.157143,0.036082,0.720280,0.136842,0.125000,0.225352


Standardize values

In [200]:
# features = df.drop(columns=["state", "year"])
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)
# df = pd.concat([df[["state", "year"]], pd.DataFrame(scaled_features, columns=features.columns)], axis=1)

Predict clusters

In [201]:
# Drop the state column
state = df['state']
features = df.drop(['state'], axis=1)

start_year = 2011
end_year = 2022
df['cluster'] = pd.NA

num_clusters = 2
for year in range(start_year, end_year + 1):
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    year_features = features[features['year'] == year].drop(['year'], axis=1)
    cluster = kmeans.fit_predict(year_features)
    df.loc[df['year'] == year, 'cluster'] = cluster
    
    print(f"Silhouette score for {year}: {silhouette_score(year_features, cluster)}")
    print(f"Inertia: {kmeans.inertia_}")
    
df

Silhouette score for 2011: 0.4250494812935507
Inertia: 15.035107732632484
Silhouette score for 2012: 0.42960998016998
Inertia: 14.991717008677899
Silhouette score for 2013: 0.42691529684299473
Inertia: 14.859525590686562
Silhouette score for 2014: 0.42523273453903915
Inertia: 15.138272634234202
Silhouette score for 2015: 0.4238739701298212
Inertia: 15.184397038620535
Silhouette score for 2016: 0.41723682204312207
Inertia: 15.329816045481255
Silhouette score for 2017: 0.4006861603096964
Inertia: 15.53377875622995
Silhouette score for 2018: 0.31093617192771883
Inertia: 15.44900939730558
Silhouette score for 2019: 0.2992956160698767
Inertia: 15.449610864464837
Silhouette score for 2020: 0.2817999076883719
Inertia: 15.43228159531563
Silhouette score for 2021: 0.19547633356875768
Inertia: 14.537027705806343
Silhouette score for 2022: 0.3407838294999389
Inertia: 13.915549069005813


,state,year,"agriculture, forestry, fishing and hunting, and mining",construction,manufacturing,wholesale trade,retail trade,"transportation and warehousing, and utilities",information,"finance and insurance, and real estate and rental and leasing","professional, scientific, and management, and administrative and waste management services","educational services, and health care and social assistance","arts, entertainment, and recreation, and accommodation and food services","other services, except public administration",public administration,cluster
0,alabama,2010,0.142857,0.803030,0.748603,0.787879,0.806452,0.370968,0.25000,0.328571,0.144330,0.440559,0.031579,0.196429,0.161972,<NA>
1,alabama,2011,0.142857,0.742424,0.731844,0.727273,0.806452,0.354839,0.21875,0.314286,0.154639,0.482517,0.036842,0.214286,0.176056,1
2,alabama,2012,0.134921,0.696970,0.709497,0.696970,0.817204,0.370968,0.21875,0.300000,0.159794,0.503497,0.047368,0.214286,0.183099,1
3,alabama,2013,0.134921,0.651515,0.703911,0.636364,0.806452,0.354839,0.18750,0.300000,0.164948,0.538462,0.057895,0.214286,0.183099,1
4,alabama,2014,0.134921,0.606061,0.709497,0.636364,0.817204,0.338710,0.18750,0.300000,0.164948,0.545455,0.068421,0.214286,0.190141,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,wyoming,2018,0.904762,0.818182,0.167598,0.454545,0.698925,0.532258,0.12500,0.071429,0.041237,0.636364,0.194737,0.071429,0.211268,1
672,wyoming,2019,0.865079,0.848485,0.167598,0.424242,0.698925,0.516129,0.12500,0.085714,0.046392,0.685315,0.184211,0.107143,0.190141,1
673,wyoming,2020,0.825397,0.848485,0.178771,0.424242,0.677419,0.532258,0.15625,0.142857,0.030928,0.741259,0.147368,0.107143,0.197183,1
674,wyoming,2021,0.793651,0.863636,0.173184,0.393939,0.731183,0.516129,0.12500,0.157143,0.036082,0.720280,0.136842,0.125000,0.225352,0


In [202]:
NEWNAME = 'State_Data_Clustered.csv'
cols = df.columns.tolist()
df = df[[cols[0]] + [cols[-1]] + cols[1:-1]]
df.to_csv('../udataset/' + NEWNAME, index=False)
print(df[(df['cluster'] == 0) & (df['year'] == 2013)]['state'])

367    nevada
Name: state, dtype: object
